<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/Expenditures_2009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pandas as pd


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-15 15:59:31--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.13MB/s    in 0.2s    

2021-09-15 15:59:32 (4.13 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/2009_expenditure.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("2009_expenditure.csv"), sep=",", header=True)
df.show()

+--------+--------+------+--------+--------+--------------+------+---------+------+--------+------+----------+--------+-------------------+-------------------+--------------------+------+-------+-----+-------------+---------+------------+-----+-----+----------+-------+---------+----------------+--------------------+--------+------+-------+
|ELECTION|OFFICECD|CANDID|CANCLASS|CANDLAST|     CANDFIRST|CANDMI|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|           INV_DATE|               DATE|                NAME|C_CODE|ORG_IND|STRNO|      STRNAME|APARTMENT|        CITY|STATE|  ZIP|PAY_METHOD|   AMNT|PURPOSECD|         PURPOSE|             EXPLAIN|EXEMPTCD|RR_IND|SEG_IND|
+--------+--------+------+--------+--------+--------------+------+---------+------+--------+------+----------+--------+-------------------+-------------------+--------------------+------+-------+-----+-------------+---------+------------+-----+-----+----------+-------+---------+----------------+--------------------

In [5]:
df.columns

['ELECTION',
 'OFFICECD',
 'CANDID',
 'CANCLASS',
 'CANDLAST',
 'CANDFIRST',
 'CANDMI',
 'COMMITTEE',
 'FILING',
 'SCHEDULE',
 'PAGENO',
 'SEQUENCENO',
 'REFNO',
 'INV_DATE',
 'DATE',
 'NAME',
 'C_CODE',
 'ORG_IND',
 'STRNO',
 'STRNAME',
 'APARTMENT',
 'CITY',
 'STATE',
 'ZIP',
 'PAY_METHOD',
 'AMNT',
 'PURPOSECD',
 'PURPOSE',
 'EXPLAIN',
 'EXEMPTCD',
 'RR_IND',
 'SEG_IND']

In [6]:
# Remove multiple columns
list = ['CANDMI','COMMITTEE',
 'FILING',
 'SCHEDULE',
 'PAGENO',
 'SEQUENCENO',
 'REFNO',
 'INV_DATE',
 'DATE',
 'NAME',
 'ORG_IND',
 'STRNO',
 'STRNAME',
 'APARTMENT',
  'EXPLAIN',
 'EXEMPTCD',
 'RR_IND',
 'SEG_IND',
 'PAY_METHOD'] 
df = df.drop(*list)
df.show()

+--------+--------+------+--------+--------+--------------+------+------------+-----+-----+-------+---------+----------------+
|ELECTION|OFFICECD|CANDID|CANCLASS|CANDLAST|     CANDFIRST|C_CODE|        CITY|STATE|  ZIP|   AMNT|PURPOSECD|         PURPOSE|
+--------+--------+------+--------+--------+--------------+------+------------+-----+-----+-------+---------+----------------+
|    2009|       5|   821|       P|   Vacca|         James|  CORP|       Bronx|   NY|10471|1235.00|    OTHER|Other: explntion|
|    2009|       5|  1164|       P|  Lander|          Brad|  CORP|  Washington|   DC|20001| 649.39|    CMAIL|    Campgn Mlngs|
|    2009|       5|    60|       P|   Behar|Steven Anthony|  CORP|Chesterbrook|   PA|19087|  17.47|    OFFCE| Office Expenses|
|    2009|       5|  1246|       P| Leandre|       Jacques|  OTHR|Chesterbrook|   PA|19087|  29.97|    OTHER|Other: explntion|
|    2009|       5|   405|       P|Hackshaw|          Rock|  CORP|        null| null| null|  29.97|    PROFL|  

In [7]:
#Combine the Candidate first and last name columns
from pyspark.sql.functions import concat_ws
df2 = df.withColumn("CandidateName", concat_ws(",","CANDFIRST",'CANDLAST'))
df2.show()

+--------+--------+------+--------+--------+--------------+------+------------+-----+-----+-------+---------+----------------+--------------------+
|ELECTION|OFFICECD|CANDID|CANCLASS|CANDLAST|     CANDFIRST|C_CODE|        CITY|STATE|  ZIP|   AMNT|PURPOSECD|         PURPOSE|       CandidateName|
+--------+--------+------+--------+--------+--------------+------+------------+-----+-----+-------+---------+----------------+--------------------+
|    2009|       5|   821|       P|   Vacca|         James|  CORP|       Bronx|   NY|10471|1235.00|    OTHER|Other: explntion|         James,Vacca|
|    2009|       5|  1164|       P|  Lander|          Brad|  CORP|  Washington|   DC|20001| 649.39|    CMAIL|    Campgn Mlngs|         Brad,Lander|
|    2009|       5|    60|       P|   Behar|Steven Anthony|  CORP|Chesterbrook|   PA|19087|  17.47|    OFFCE| Office Expenses|Steven Anthony,Behar|
|    2009|       5|  1246|       P| Leandre|       Jacques|  OTHR|Chesterbrook|   PA|19087|  29.97|    OTHER|Oth

In [8]:
#Rearrange column order
df3=df2.select('ELECTION','OFFICECD', 'CANDID', 'CANCLASS', 'CandidateName', 'CANDLAST', 'CANDFIRST','C_CODE', 'CITY', 'STATE',  'ZIP', 'AMNT', 'PURPOSECD', 'PURPOSE')

In [9]:
#Drop OFFICECD and CANCLASS
list1 = ['CANDLAST', 'CANDFIRST'] 
df4 = df3.drop(*list1)
df4.show()

+--------+--------+------+--------+--------------------+------+------------+-----+-----+-------+---------+----------------+
|ELECTION|OFFICECD|CANDID|CANCLASS|       CandidateName|C_CODE|        CITY|STATE|  ZIP|   AMNT|PURPOSECD|         PURPOSE|
+--------+--------+------+--------+--------------------+------+------------+-----+-----+-------+---------+----------------+
|    2009|       5|   821|       P|         James,Vacca|  CORP|       Bronx|   NY|10471|1235.00|    OTHER|Other: explntion|
|    2009|       5|  1164|       P|         Brad,Lander|  CORP|  Washington|   DC|20001| 649.39|    CMAIL|    Campgn Mlngs|
|    2009|       5|    60|       P|Steven Anthony,Behar|  CORP|Chesterbrook|   PA|19087|  17.47|    OFFCE| Office Expenses|
|    2009|       5|  1246|       P|     Jacques,Leandre|  OTHR|Chesterbrook|   PA|19087|  29.97|    OTHER|Other: explntion|
|    2009|       5|   405|       P|       Rock,Hackshaw|  CORP|        null| null| null|  29.97|    PROFL|    Prof. Srvcs.|
|    200

In [10]:
# check data types
df4.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('CANDID', 'string'),
 ('CANCLASS', 'string'),
 ('CandidateName', 'string'),
 ('C_CODE', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('AMNT', 'string'),
 ('PURPOSECD', 'string'),
 ('PURPOSE', 'string')]

In [11]:
df4.columns

['ELECTION',
 'OFFICECD',
 'CANDID',
 'CANCLASS',
 'CandidateName',
 'C_CODE',
 'CITY',
 'STATE',
 'ZIP',
 'AMNT',
 'PURPOSECD',
 'PURPOSE']

In [12]:
#Change column names
df5 = df4 \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("CANDID", "CandidateID") \
.withColumnRenamed("C_CODE", "ContributionType") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("PURPOSECD", "PurposeCode") \
.withColumnRenamed("PURPOSE", "Purpose") 

df5.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CandidateID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = false)
 |-- ContributionType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- PurposeCode: string (nullable = true)
 |-- Purpose: string (nullable = true)



In [13]:
# changing datatypes of columns
df6 = df5 \
  .withColumn("Election",df5["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df5["OFFICECD"].cast("Integer")) \
  .withColumn("CandidateID", df5["CandidateID"].cast("Integer")) \
  .withColumn("ZipCode", df5["ZipCode"].cast("Integer")) \
  .withColumn("Amount", df5["Amount"].cast("Float")) \
  
df6.printSchema()

root
 |-- Election: integer (nullable = true)
 |-- OFFICECD: integer (nullable = true)
 |-- CandidateID: integer (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = false)
 |-- ContributionType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- Amount: float (nullable = true)
 |-- PurposeCode: string (nullable = true)
 |-- Purpose: string (nullable = true)



In [14]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df7 = df6.withColumn("ContributionType", regexp_replace("ContributionType", 'CAN', 'Candidate')) \
  .withColumn('ContributionType', regexp_replace("ContributionType", 'CORP', 'Corporation')) \
  .withColumn('ContributionType', regexp_replace("ContributionType", 'EMPO', 'Labor Union')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'FAM', 'Candidate Family')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'IND', 'Individual')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PCOMZ', 'Party Committee')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'ORG', 'Orgainization')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'LLC', 'Limited Liability Company')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PCOMP', 'Political Action Committee')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'SPO', ' Candidates Spouse')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'OTHR', 'Other')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PCOMC', 'Candidate Committee')) \
  .withColumn("ContributionType", regexp_replace("ContributionType", 'PART', 'Individual')) \


In [15]:
#Change Purpose Code value names
from pyspark.sql.functions import regexp_replace
df8 = df7.withColumn('PurposeCode', regexp_replace('PurposeCode', 'ADVAN', 'Advance Repayment')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CMAIL', 'Campaign Mail')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CMISC', 'Miscellaneous')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CNTRB', 'Political Contributions')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'COMPL', 'Compliance Cost')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CONSL', 'Campaign Consultants')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'CONSV', 'Constituent Services')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'FUNDR', 'Fundraising')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'INTER', 'Interest Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'LITER', 'Campaign Literature')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'NQUAL', 'Non Qualified Expenditures')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'OFFCE', 'Office Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'OTHER', 'Other')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'PETIT', 'Petition Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'POLLS', 'Polling Costs')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'POSTA', 'Postal Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'PRINT', 'Printing Expenses')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'PROFL', 'Professional Services')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'RADIO', 'Radio Ads')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'RENTO', 'Office Rent')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'TVADS', 'Television Ads')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'UNKN', 'Unknown')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'VOTER', 'Voter Registration')) \
  .withColumn('PurposeCode', regexp_replace('PurposeCode', 'WAGES', 'Campaign Workers Salaries')) \

In [16]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df9 = df8.filter((df6.OFFICECD=="1") & (df8.CANCLASS=="P"))
df9.show()

+--------+--------+-----------+--------+--------------------+--------------------+-------------+-----+-------+-------+-----------------+----------------+
|Election|OFFICECD|CandidateID|CANCLASS|       CandidateName|    ContributionType|         City|State|ZipCode| Amount|      PurposeCode|         Purpose|
+--------+--------+-----------+--------+--------------------+--------------------+-------------+-----+-------+-------+-----------------+----------------+
|    2009|       1|        260|       P|William,Thompson,...|         Corporation|     New York|   NY|  10007|  38.55|            Other|Other: explntion|
|    2009|       1|        260|       P|William,Thompson,...|         Corporation|     New York|   NY|  10007|   49.0|            Other|Other: explntion|
|    2009|       1|       1286|       P|        Kevin,Coenen|Limited Liability...|     New York|   NY|  10106|   30.0|            Other|Other: explntion|
|    2009|       1|       1286|       P|        Kevin,Coenen|Limited Liabili

In [17]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df9.select('OFFICECD').distinct().show()

+--------+
|OFFICECD|
+--------+
|       1|
+--------+



In [18]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
Expenditures_2009_df = df9.drop(*list2)
Expenditures_2009_df.show()

+--------+-----------+--------------------+--------------------+-------------+-----+-------+-------+-----------------+----------------+
|Election|CandidateID|       CandidateName|    ContributionType|         City|State|ZipCode| Amount|      PurposeCode|         Purpose|
+--------+-----------+--------------------+--------------------+-------------+-----+-------+-------+-----------------+----------------+
|    2009|        260|William,Thompson,...|         Corporation|     New York|   NY|  10007|  38.55|            Other|Other: explntion|
|    2009|        260|William,Thompson,...|         Corporation|     New York|   NY|  10007|   49.0|            Other|Other: explntion|
|    2009|       1286|        Kevin,Coenen|Limited Liability...|     New York|   NY|  10106|   30.0|            Other|Other: explntion|
|    2009|       1286|        Kevin,Coenen|Limited Liability...|     New York|   NY|  10106|   30.0|            Other|Other: explntion|
|    2009|        260|William,Thompson,...|     

In [22]:
#Export to new CSV
Expenditures_2009_df.toPandas().to_csv("Clean_Expenditures_2009.csv", header=True)

In [20]:
# Store environmental variable
from getpass import getpass
#Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-final.cjdbapst3wjf.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "88888",
          "driver":"org.postgresql.Driver"}

In [21]:
# Write DataFrame to active_user table in RDS
#Expenditures_2009_df.write.jdbc(url=jdbc_url, table='expenditures_2009', mode=mode, properties=config)